## YOLO

In [ ]:
!pip install ultralytics

In [ ]:
import base64
import PIL.Image as PImage

from io import BytesIO
from ultralytics import YOLO
from huggingface_hub import hf_hub_download

In [ ]:
# model_path = hf_hub_download(repo_id="arnabdhar/YOLOv8-Face-Detection", filename="model.pt")
model_path = hf_hub_download(repo_id="AdamCodd/YOLOv11n-face-detection", filename="model.pt")

detection_model = YOLO(model_path)

In [ ]:
# from PIL
img = PImage.open("people.jpg").convert("RGB")
iw, ih = img.size
img = img.resize((800, int(800/iw*ih)))
display(img)

In [ ]:
# from base64
with open("./people.jpg", "rb") as inf:
  img64 = f"data:image/jpeg;base64,{base64.b64encode(inf.read()).decode('utf-8')}"

image_data = base64.b64decode(img64.split("base64,")[1])
img = PImage.open(BytesIO(image_data))
iw, ih = img.size
img = img.resize((800, int(800/iw*ih)))
display(img)

In [ ]:
output = detection_model.predict(img, verbose=False)

In [ ]:
faces =[]

for out in output:
  for bbox,conf in zip(out.boxes.xywhn, out.boxes.conf):
    faces.append({
      "xywhn": bbox.tolist(),
      "conf": conf.item()
    })
len(faces)

## dlib

https://github.com/davisking/dlib

In [ ]:
!pip install dlib

In [ ]:
!wget http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_5_face_landmarks.dat.bz2

In [ ]:
import dlib
import numpy as np
from PIL import Image as PImage

In [ ]:
predictor_path = "./shape_predictor_5_face_landmarks.dat"
face_file_path = "./people.jpg"

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

img = np.array(PImage.open(face_file_path))

# upsample by 1
dets = detector(img, 1)

num_faces = len(dets)
if num_faces == 0:
  print("Sorry, there were no faces found in '{}'".format(face_file_path))

num_faces

In [ ]:
# Find the 5 face landmarks we need to do the alignment.
faces = [predictor(img, detection) for detection in dets]
len(faces)

In [ ]:
images = dlib.get_face_chips(img, dlib.full_object_detections(faces), size=170, padding=0.35)

for im in images:
  display(PImage.fromarray(im).crop((20,0,150,170)))